In [1]:
import os
import sys
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Jupyter Magic to make plots show up inline
%matplotlib inline

import gymnasium as gym
import gym4real
from gymnasium import spaces
from gym4real.envs.wds.utils import parameter_generator
from stable_baselines3 import DQN, A2C
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from sb3_contrib import ARS

import wntr
import wntr.sim

# Force the Python simulator to prevent M3 Kernel Crashes
print("🔧 Patching WNTR Simulator for M3 Compatibility...")
wntr.sim.EpanetSimulator = wntr.sim.WNTRSimulator

🔧 Patching WNTR Simulator for M3 Compatibility...


/Users/kavish/miniforge3/envs/rl/lib/python3.10/site-packages/wntr/epanet/toolkit.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
config_path = os.path.join(os.getcwd(), "gym4real", "envs", "wds", "world_anytown.yaml")

DURATION_STEPS = 168 

base_params = parameter_generator(
    world_options=config_path, 
    hydraulic_step=3600,       # 1 hour steps
    duration=3600 * 24 * 7,    # 7 days in seconds
    seed=42)

def make_env():
    params = copy.deepcopy(base_params)
    env = gym.make('gym4real/wds-v0', settings=params)
        
    return env

In [3]:
env = make_env()
obs, info = env.reset()

print("--- The Real Data (27 Values) ---")
print(obs)

# Breakdown
print(f"\nTank Levels: {obs[0:2]}")
print(f"Pressures:   {obs[2:24]}")
print(f"Demand:      {obs[24]}")
print(f"Time:        {obs[25:]}")

env.close()

Resetting the environment...
--- The Real Data (27 Values) ---
[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.14 0.   1.  ]

Tank Levels: [0. 0.]
Pressures:   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Demand:      0.13987360894680023
Time:        [0. 1.]


/Users/kavish/miniforge3/envs/rl/lib/python3.10/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/Users/kavish/miniforge3/envs/rl/lib/python3.10/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [4]:
# CREATE TRAINING ENVIRONMENT
train_env = DummyVecEnv([lambda: make_env()])
train_env = VecMonitor(train_env)

MODEL_NAME = "DQN_WDSEnv"

print("🧠 Initializing DQN Agent...")
model = DQN(
    "MlpPolicy", 
    train_env,
    device='cpu',
    verbose=1)

print(f"Training {MODEL_NAME}...")
model.learn(total_timesteps=200000) # Increase this for better results
print("✅ Training Complete.")

# Save the model
model.save(MODEL_NAME)

/Users/kavish/miniforge3/envs/rl/lib/python3.10/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/Users/kavish/miniforge3/envs/rl/lib/python3.10/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


🧠 Initializing DQN Agent...
Using cpu device
Training DQN_WDSEnv...
Resetting the environment...
Resetting the environment...
Resetting the environment...
Resetting the environment...
Resetting the environment...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 223      |
|    ep_rew_mean      | 168      |
|    exploration_rate | 0.958    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 34       |
|    time_elapsed     | 25       |
|    total_timesteps  | 893      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.148    |
|    n_updates        | 198      |
----------------------------------
Resetting the environment...
Resetting the environment...
Resetting the environment...
Resetting the environment...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 224      |
|    ep_rew_mean      | 174      |
|    e

/Users/kavish/Documents/gym4real/gym4real/envs/wds/simulator/epynet/epanet2.py:683: UserWarning: WARNING: Pumps cannot deliver enough flow or head.
  warnings.warn(self.ENgeterror(ierr))


Resetting the environment...
Resetting the environment...
Resetting the environment...
Resetting the environment...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 225      |
|    ep_rew_mean      | 150      |
|    exploration_rate | 0.872    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 34       |
|    time_elapsed     | 77       |
|    total_timesteps  | 2704     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.194    |
|    n_updates        | 650      |
----------------------------------
Resetting the environment...
Resetting the environment...
Resetting the environment...
Resetting the environment...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 227      |
|    ep_rew_mean      | 154      |
|    exploration_rate | 0.828    |
| time/               |          |
|    episodes         | 16       

/Users/kavish/Documents/gym4real/gym4real/envs/wds/simulator/epynet/epanet2.py:683: UserWarning: WARNING: Pumps cannot deliver enough flow or head.
  warnings.warn(self.ENgeterror(ierr))


...

Resetting the environment...
Resetting the environment...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 190      |
|    ep_rew_mean      | 134      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 160      |
|    fps              | 33       |
|    time_elapsed     | 965      |
|    total_timesteps  | 31989    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0596   |
|    n_updates        | 7972     |
----------------------------------
Resetting the environment...
Resetting the environment...
Resetting the environment...
Resetting the environment...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 190      |
|    ep_rew_mean      | 133      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 164      |
|    fps              | 33       |
|    time_elapsed    

/Users/kavish/Documents/gym4real/gym4real/envs/wds/simulator/epynet/epanet2.py:683: UserWarning: WARNING: Pumps cannot deliver enough flow or head.
  warnings.warn(self.ENgeterror(ierr))


Resetting the environment...
Resetting the environment...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 190      |
|    ep_rew_mean      | 134      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 196      |
|    fps              | 29       |
|    time_elapsed     | 1321     |
|    total_timesteps  | 38950    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.029    |
|    n_updates        | 9712     |
----------------------------------
Resetting the environment...
Resetting the environment...
Resetting the environment...


/Users/kavish/Documents/gym4real/gym4real/envs/wds/simulator/epynet/epanet2.py:683: UserWarning: WARNING: Pumps cannot deliver enough flow or head.
  warnings.warn(self.ENgeterror(ierr))


Resetting the environment...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 190      |
|    ep_rew_mean      | 133      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 200      |
|    fps              | 29       |
|    time_elapsed     | 1343     |
|    total_timesteps  | 39727    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0225   |
|    n_updates        | 9906     |
----------------------------------
Resetting the environment...
Resetting the environment...
Resetting the environment...
Resetting the environment...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 191      |
|    ep_rew_mean      | 136      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 204      |
|    fps              | 29       |
|    time_elapsed     | 1368     |
|    total_time

In [5]:
# CREATE TRAINING ENVIRONMENT
train_env = DummyVecEnv([lambda: make_env()])
train_env = VecMonitor(train_env)

MODEL_NAME = "A2C_WDSEnv"

print("🧠 Initializing A2C Agent...")
model = A2C(
    "MlpPolicy", 
    train_env, 
    learning_rate=1e-4,
    device="cpu", 
    verbose=1)

print(f"Training {MODEL_NAME}...")
model.learn(total_timesteps=200000) # Increase this for better results
print("✅ Training Complete.")

# Save the model
model.save(MODEL_NAME)

/Users/kavish/miniforge3/envs/rl/lib/python3.10/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/Users/kavish/miniforge3/envs/rl/lib/python3.10/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


🧠 Initializing A2C Agent...
Using cpu device
Training A2C_WDSEnv...
Resetting the environment...
Resetting the environment...
Resetting the environment...
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 216      |
|    ep_rew_mean        | 198      |
| time/                 |          |
|    fps                | 33       |
|    iterations         | 100      |
|    time_elapsed       | 14       |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | -0.00343 |
|    learning_rate      | 0.0001   |
|    n_updates          | 99       |
|    policy_loss        | 3.72     |
|    value_loss         | 9.62     |
------------------------------------
Resetting the environment...
Resetting the environment...
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 226      |
|    ep_rew_mean        | 184      |
| time/   

/Users/kavish/Documents/gym4real/gym4real/envs/wds/simulator/epynet/epanet2.py:683: UserWarning: WARNING: Pumps cannot deliver enough flow or head.
  warnings.warn(self.ENgeterror(ierr))


Resetting the environment...
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 225      |
|    ep_rew_mean        | 181      |
| time/                 |          |
|    fps                | 34       |
|    iterations         | 2900     |
|    time_elapsed       | 419      |
|    total_timesteps    | 14500    |
| train/                |          |
|    entropy_loss       | -1.33    |
|    explained_variance | -33.1    |
|    learning_rate      | 0.0001   |
|    n_updates          | 2899     |
|    policy_loss        | 4.24     |
|    value_loss         | 29.5     |
------------------------------------
Resetting the environment...
Resetting the environment...
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 226       |
|    ep_rew_mean        | 182       |
| time/                 |           |
|    fps                | 34        |
|    iterations         | 3000      |
|    time_elapsed 

/Users/kavish/Documents/gym4real/gym4real/envs/wds/simulator/epynet/epanet2.py:683: UserWarning: WARNING: Pumps cannot deliver enough flow or head.
  warnings.warn(self.ENgeterror(ierr))


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 227      |
|    ep_rew_mean        | 185      |
| time/                 |          |
|    fps                | 34       |
|    iterations         | 4500     |
|    time_elapsed       | 653      |
|    total_timesteps    | 22500    |
| train/                |          |
|    entropy_loss       | -1.35    |
|    explained_variance | 0        |
|    learning_rate      | 0.0001   |
|    n_updates          | 4499     |
|    policy_loss        | 2.32     |
|    value_loss         | 4.76     |
------------------------------------
Resetting the environment...
Resetting the environment...
Resetting the environment...
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 227      |
|    ep_rew_mean        | 185      |
| time/                 |          |
|    fps                | 34       |
|    iterations         | 4600     |
|    time_elapsed       |

/Users/kavish/Documents/gym4real/gym4real/envs/wds/simulator/epynet/epanet2.py:683: UserWarning: WARNING: Pumps cannot deliver enough flow or head.
  warnings.warn(self.ENgeterror(ierr))


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 227      |
|    ep_rew_mean        | 186      |
| time/                 |          |
|    fps                | 34       |
|    iterations         | 4800     |
|    time_elapsed       | 696      |
|    total_timesteps    | 24000    |
| train/                |          |
|    entropy_loss       | -1.24    |
|    explained_variance | 2.44e-06 |
|    learning_rate      | 0.0001   |
|    n_updates          | 4799     |
|    policy_loss        | 1.43     |
|    value_loss         | 1.12     |
------------------------------------
Resetting the environment...
Resetting the environment...
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 228      |
|    ep_rew_mean        | 186      |
| time/                 |          |
|    fps                | 34       |
|    iterations         | 4900     |
|    time_elapsed       | 711      |
|    total_timest